Install the tweepy library

In [1]:
# !pip install tweepy
# !pip install mysqlclient

Import required libraries

In [2]:
import os
import re
import time
import re

import tweepy as tw
import pandas as pd
from geopandas.tools import geocode
import numpy as np
from datetime import datetime, timedelta, timezone
import MySQLdb


# For sentiment analysis of tweets
from textblob import TextBlob

Create Tweeter Consumer Keys and Access Tokens 

In [3]:
consumer_key= 'NFIO7bIgwQpdCZ7p7rgewKXss'
consumer_secret= '2QegNrIpcGCW6nCWFTL5JEsrDCmYZpQHJ8SdIT7V0ql5cQgFwV'
access_token= '3049227557-36ln3qak9xbxHvgNos45cf3uScGxuUdaDAR4vqs'
access_token_secret= 'pZ4Y0p31graKrAYAy41mYztvIrYFzof7awiTDWQr2ui9u'

Initialize AuthHandler

In [4]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

Define search words and date

In [5]:
search_words = "#Covid"
date_since = "2018-11-16"

In [6]:
tweets = tw.Cursor(api.search, q=search_words, lang="en", since=date_since).items(5)

In [7]:
tweets

Iterate through tweets using for loop

In [8]:
from datetime import datetime, timedelta
from email.utils import parsedate_tz

s='Mon Dec 07 17:03:03 +0000 2020'
def to_datetime(datestring):
    time_tuple = parsedate_tz(datestring.strip())
    dt = datetime(*time_tuple[:6])
    proper_date=(dt - timedelta(seconds=time_tuple[-1]))
    return proper_date

print(to_datetime(s))

dt_string = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(dt_string)

2020-12-07 17:03:03
2021-03-19 19:14:28


Retrieve tweets as a collection of list

In [9]:
# [tweet.text for tweet in tweets]

Search Tweets without Retweets

In [10]:
new_search = search_words 
# + " -filter:retweets"

In [11]:
new_search

'#Covid'

In [12]:
tweets = tw.Cursor(api.search, q=new_search, lang="en",since=date_since).items(10)

In [13]:
[tweet.text for tweet in tweets]

['RT @vineetbhola90: Germany allowed its states to open their schools and KGs from mid Feb (CW7 for Saxony, CW8 for some other states)\n5-9 Y…',
 '#MIT Engineers Develop a Cheap Ventilator for COVID-19 Treatment https://t.co/tuCWX5UglJ #covid #design #event',
 'RT @vineetbhola90: Germany allowed its states to open their schools and KGs from mid Feb (CW7 for Saxony, CW8 for some other states)\n5-9 Y…',
 'RT @DrNinaRadcliff: In #Israel they have a #vaccine #passport that allows #vaccinated people access to #gyms #restaurants #hotels #concerts…',
 'RT @TorontoWriter: Dr Coleman listing deaths by experimental #Covid injection shot.\n\n#coronavirus #health #medtwitter #vaxxed #lockdowns #p…',
 'RT @drrajivguptaias: .@AmdavadAMC will start super spreaders’ #COVID testing at multiple locations in city from tomorrow.Home delivery staf…',
 'RT @AZKARKOUR: First #business trip in #COVID times since March 2020 leading me to #Romania with a magic drive in snow from #Bucharest to c…',
 'RT @BenMart

Get username and location

In [14]:
tweets = tw.Cursor(api.search,q=new_search, lang="en", since=date_since).items(20)

In [15]:
  # clean text
text_preprocess = lambda x: re.compile('\#').sub('', re.compile('RT @').sub('@', x).strip())

In [16]:
# created_at=to_datetime(all_data["created_at"])
#             created_at_date=created_at.date()

users_locs = [[tweet.user.screen_name,tweet.user.name,tweet.user.verified,
         tweet.user.followers_count,tweet.user.friends_count,tweet.user.listed_count,
         tweet.retweet_count,tweet.favorite_count,tweet.retweeted,tweet.entities,
         tweet.user.favourites_count,
         tweet.user.location,tweet.created_at,tweet.text,
         re.sub(r"http\S+", "", re.sub('@[^\s]+','',text_preprocess(tweet.text))),
         TextBlob(re.sub(r"http\S+", "", re.sub('@[^\s]+','',text_preprocess(tweet.text)))).sentiment[0],
         TextBlob(re.sub(r"http\S+", "", re.sub('@[^\s]+','',text_preprocess(tweet.text)))).sentiment[1]
              ] for tweet in tweets]

In [17]:
# users_locs

In [18]:
cols=columns=['screen_name','name','user_verification','followers_count','friends_count',
              'listed_count','retweet_count','favorite_count','retweeted','entities','favourites_count',
              'location','created_at','text','clean_text','sentiment_polarity','sentiment_subjectivity']

Create a Pandas Dataframe From A List of Tweet Data

In [19]:
tweet_df = pd.DataFrame(data=users_locs, columns=cols)

Create a color column based on sentiment

In [20]:
tweet_df["sentiment_polarity_color"] = np.where(tweet_df["sentiment_polarity"]<0, 'red', 'green')

Enrich data with geo-coordinates from location names

In [21]:
# def custom_geocoder(address):
#     try:
#         dataframe = geocode(address.strip() , provider="nominatim" , user_agent = 'my_request')
#         point = dataframe.geometry.iloc[0]
#         return pd.Series({'Latitude': point.y, 'Longitude': point.x})
#     except:
#         return None

In [22]:
# tweet_df[['latitude' , 'longitude']]= tweet_df.iloc[:,12].apply( lambda x: custom_geocoder(x))

tweet_df.head()

In [24]:
tweet_df.to_csv("tweets.csv")

Tokenization Function

In [25]:
def text_tokenizaion(text):
    return re.split(' ',text)

In [26]:
sample_text="The current pandemic shows that we can 'no longer ignore the risks"
text_tokenizaion(sample_text)

['The',
 'current',
 'pandemic',
 'shows',
 'that',
 'we',
 'can',
 "'no",
 'longer',
 'ignore',
 'the',
 'risks']

## Working With The Twitter Streaming API

With Streaming data we need to do the following:
1. Create database connection
2. Create a persistent connection to the Twitter API, and read each connection incrementally.
3. Process tweets quickly, store them in the database, and don’t let your program get backed up.
4. Handle errors and other issues properly.

Connect to MySql Database

In [27]:
#                           (server,MySQL username,MySQL pass, Database name).
connection = MySQLdb.connect("localhost","root","root","social_analytic")
cur = connection.cursor()

OperationalError: (1045, "Access denied for user 'root'@'localhost' (using password: YES)")

StreamListener Class

In [ ]:
from tweepy import Stream
from tweepy.streaming import StreamListener
import tweepy
import time
import json
import sys


search_val="covid"

class listener(StreamListener):

    def on_data(self, data):
       
        try:
#             tweet_count=0
            
            all_data = json.loads(data)
            
#             search_val="covid"
            tweet_id=all_data["id"]
            created_at=to_datetime(all_data["created_at"])
            created_at_date=created_at.date()
            created_at_hour=created_at.time().strftime("%H")
            created_at_min=created_at.time().strftime("%H:%M")
            text = all_data["text"]
            source = all_data["source"]
#             local_created_at=created_at.str.replace(tzinfo=timezone.utc).astimezone(tz=None)
            local_created_at=all_data["created_at"]
        
        
            in_reply_to_status_id=all_data['in_reply_to_status_id_str']
            in_reply_to_user_id=all_data['in_reply_to_user_id_str']
            in_reply_to_screen_name=all_data['in_reply_to_screen_name']
            username=all_data['user']['name']
            user_screen_name=all_data['user']['screen_name']
            user_location=all_data['user']['location']
            user_url=all_data['user']['url']
            user_description=all_data['user']['description']
            user_verified_str=all_data['user']['verified']
            user_verified=0
            if user_verified_str==True:
                user_verified=1
            user_followers_count=all_data['user']['followers_count']
            user_friends_count=all_data['user']['friends_count']
            user_listed_count=all_data['user']['listed_count']
            user_favourites_count=all_data['user']['favourites_count']
            user_created_at=to_datetime(all_data['user']['created_at'])
            user_created_at_date=user_created_at.date()
            user_id=all_data['user']['id']
            user_profile_image_url_https=all_data['user']['profile_image_url_https']
            
#             coordinates_lat=all_data['coordinates'][0]
#             coordinates_lon=all_data['coordinates'][1]
            if all_data['coordinates']==True:
                coordinates_lat=all_data['coordinates'][0]
                coordinates_lon=all_data['coordinates'][1]
            else:
                coordinates_lat=None
                coordinates_lon=None
            if all_data['place']==True:
                place_country=all_data['place_country']
                place_country_code=all_data['place_country_code']
                place_full_name=all_data['place_full_name']
                place_id=all_data['place_id']
                place_type=all_data['place_type']
            else:
                place_country=None
                place_country_code=None
                place_full_name=None
                place_id=None
                place_type=None
                
            retweeted_status=None
            
            if all_data['is_quote_status']==True:
                quoted_status_id=all_data['quoted_status_id']
                quoted_status=all_data['quoted_status']
#                 retweeted_status=all_data['retweeted_status']
                quote_count=0
            else:
                quoted_status_id=None
                quote_count=0
                quoted_status=None
#                 retweeted_status=None
            try:
                reply_count=all_data['reply_count']
            except:
                reply_count=0
            retweet_count=all_data['retweet_count']
            favorite_count=all_data['favorite_count']
            retweeted_txt=all_data['retweeted']
            entities=str(all_data['entities'])
            retweeted=0
            if retweeted_txt==True:
                retweeted=1
#             feeds_link=all_data['https://twitter.com/_/status/'+id]
            feeds_link=None
            lang=all_data['lang']
            feeds_image=None
            feeds_video=None
            # clean text
            text_preprocess = lambda x: re.compile('\#').sub('', re.compile('RT @').sub('@', x).strip())
            clean_text=text_preprocess(text)
            clean_text=re.sub('@[^\s]+','',clean_text)
            clean_text=re.sub(r"http\S+", "", clean_text)
            tokenized_text=text_tokenizaion(clean_text)
            # end clean text
            text_sentiment_polarity=TextBlob(clean_text).sentiment[0]
            text_sentiment_subjectivity=TextBlob(clean_text).sentiment[1]


            cur.execute("INSERT INTO tweet (search_val,  created_at, created_at_date,created_at_hour,created_at_min, tweet_id,  text, source,in_reply_to_status_id, in_reply_to_user_id,  in_reply_to_screen_name,  user_name,  user_screen_name,  user_location, user_url,  user_description, user_verified, user_followers_count,  user_friends_count, user_listed_count,  user_favourites_count,  user_created_at,user_created_at_date, user_id,  user_profile_image_url_https,  coordinates_lat,  coordinates_lon, place_country,  place_country_code,  place_full_name, place_id,  place_type,  quoted_status_id,  quoted_status, retweeted_status,  quote_count,  reply_count,  retweet_count,  favorite_count, retweeted,  entities,  lang, feeds_link,  feeds_video, feeds_image,clean_text,text_sentiment_polarity,text_sentiment_subjectivity) VALUES (%s, %s,%s, %s,%s, %s,%s,%s,%s, %s, %s, %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", 
                        (search_val,created_at,created_at_date,created_at_hour,created_at_min,tweet_id,text,source,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,username,user_screen_name,user_location,user_url,user_description,user_verified,user_followers_count,user_friends_count,user_listed_count,user_favourites_count,user_created_at,user_created_at_date,user_id,user_profile_image_url_https,coordinates_lat,coordinates_lon,place_country,place_country_code,place_full_name,place_id,place_type,quoted_status_id,quoted_status,retweeted_status,quote_count,reply_count,retweet_count,favorite_count,retweeted,entities,lang,feeds_link,feeds_video,feeds_image,clean_text,text_sentiment_polarity,text_sentiment_subjectivity))

            connection.commit()
            print("Fetched tweets ...........\n")
            print(created_at,user_created_at)
#             tweet_count +=1

            return True

# Catch all the errors from Database or Twiteer
        except tweepy.error.TweepError:
            print("Error fetching the data")
            
        except UnicodeEncodeError:
            pass
        
        except MySQLdb.DataError as e:
            print("DataError")
            print(e)

        except MySQLdb.InternalError as e:
            print("InternalError")
            print(e)

        except MySQLdb.IntegrityError as e:
            print("IntegrityError")
            print(e)

        except MySQLdb.OperationalError as e:
            print("OperationalError")
            print(e)

        except MySQLdb.NotSupportedError as e:
            print("NotSupportedError")
            print(e)

        except MySQLdb.ProgrammingError as e:
            print("ProgrammingError")
            print(e)
        except:
            print ("Unexpected general error: ", sys.exc_info()[0])
            raise
        finally:
            print("  ")
            #             cur.close()
            #             connection.close()
            

    def on_error(self, status):
        print (status)

In [ ]:
twitterStream = Stream(auth, listener())
twitterStream.filter(track=['covid'],languages = ["en"], stall_warnings = True)